In [5]:
import basis as bs
import mesh as mm
import integrate as ig
import assembly as asm
import numpy.polynomial.legendre as lg
import numpy as np

In [6]:
class generate_mesh():
    def __init__(self, phys_xmin, phys_xmax, n_elem):
        self.phys_xmin = phys_xmin
        self.phys_xmax = phys_xmax
        self.n_elem = n_elem
        
        self.gen_all_nodes()
        self.gen_elem_node_coord_indices()

    def gen_all_nodes(self):
        self.all_nodes = np.linspace(self.phys_xmin, self.phys_xmax, self.n_elem + 1)
        return self.all_nodes
    def gen_elem_node_coord_indices(self):
        arr = np.zeros((2, self.n_elem), dtype=int)
        for n in range(0, self.n_elem):
            arr[0,n] = n
            arr[1,n] = n+1
        self.elem_node_coord_indices = arr
        return self.elem_node_coord_indices
    def get_element_domain(self, e):
        n0_idx = int(self.elem_node_coord_indices[0,e])
        n1_idx = int(self.elem_node_coord_indices[1,e])
        self.element_domain = np.array((self.all_nodes[n0_idx], self.all_nodes[n1_idx]))
        return self.element_domain

In [7]:

def gen_ke(mesh, e):
    # for e in range(0, self.n_elem):
    # mx = bs.stock_mx()
    mx_ = np.zeros((2,2))
    e_dom = mesh.get_element_domain(e)
    e_x0 = e_dom[0]
    e_x1 = e_dom[1]
    mx_mult = bs.mxm(e_x0, e_x1)
    inv_term = bs.x_xi_deriv_inv(e_x0, e_x1)
    for a in range(0,2):
        N_a_xi_term = bs.N_a_xi(a)
        for b in range(0,2):
            N_b_xi_term = bs.N_a_xi(b)
            ke_entry_incr = N_a_xi_term * N_b_xi_term * inv_term
            mx_[a, b] += ke_entry_incr
    ke = mx_
    return ke

In [8]:
m1 = generate_mesh(0, 1, 1)

e_dom = m1.get_element_domain(0)
e_x0 = e_dom[0]
e_x1 = e_dom[1]
# print(f"{e_x0}, {e_x1}")
mult = bs.mxm(e_x0, e_x1)
inv_term = bs.x_xi_deriv_inv(e_x0, e_x1)
# print(inv_term)
# gen_ke(m1, 0)
N_ab_xi_term = bs.N_a_xi(1) * bs.N_a_xi(1)
ke_entry_incr = N_ab_xi_term * inv_term

In [9]:
def N_a_xi(idx):
    neg = (-1)**idx
    return neg / 2
# THIS RETURNS POS/NEG 0.5, NOT 1

def Nab(a,b):
    return N_a_xi(a) * N_a_xi(b)
# SO WITH TWO OF THEM, RETURNS POS/NEG 0.25
# Nab(0,0) # +0.25
# Nab(0,1) # -0.25


In [10]:
# ADD IN INVERSE X-XI MAP TERM:
def x_xi_deriv_inv(e_x0, e_x1): 
    h_e = e_x1 - e_x0
    return 2 / h_e
# AND FOR ONE ELEMENT ON 0,1, WE GET:
inv_term = x_xi_deriv_inv(0,1) # 2
print(inv_term)
# SO WITH THE ABOVE, WE HAVE A PRODUCT OF POS/NEG 0.5
x_xi_deriv_inv(0,1) * Nab(0,0)
# print(prod)
# BUT THIS SHOULD COME OUT TO THE SAME AS THE STOCK MATRIX x 1/H_E

2.0


0.5

In [11]:
# BUT WE ALSO HAVE TO INTEGRATE: WE DON'T GET STOCK MATRIX x 1/H_E FROM THE ABOVE PRODUCT. WE GET IT FROM EVALUATING THAT PRODUCT'S INTEGRAL FROM -1 TO 1
prod = x_xi_deriv_inv(0,1) * Nab(0,0)
upper = prod * 1 
lower = prod * -1
upper - lower

# HOLY CRAP THAT'S IT. 
# NOW, GAUSS QUADRATURE SHOULD YIELD THE SAME THING:

1.0

In [12]:
# def prodfn(a,b):
#     return x_xi_deriv_inv(0,1) * Nab(a,b)

# # pts, wts = ig.get_gauss_quadrature(3)
# # wts

# fnxab = lambda x, a, b: prodfn(a, b)
# fnxab(0, 0,0) # 0.5, which is what we got before integrating above !

# # makeshift gauss quadrature function
# def gquad(a, b, n_quad):
#     pts, wts = ig.get_gauss_quadrature(n_quad)
#     out = 0
#     for q in range(0, n_quad):
#         pt = pts[q]
#         wt = wts[q]
#         incr = fn(pt,a,b) * wt
#         out += incr
#     return out

# gquad(0,1,1)
# # WORKS! RETURNS 1: POS FOR A=B, NEG OTHERWISE


NameError: name 'fn' is not defined

In [ ]:
# holy mama. now trying with lambda and the existing gq integration function:

def integrate_by_quadrature2(function, x_lower, x_upper, n_quad, a_in, b_in):
    # print(f"\nrunning quadrature for a={a_in}, b={b_in} ")
    quad = ig.get_gauss_quadrature(n_quad)
    gauss_points = quad[0]
    # print(f"pts={gauss_points}")
    gauss_weights = quad[1]
    # print(f"wts={gauss_weights}")
    jacobian = ig.get_jacobian(x_lower, x_upper)
    # print(f"J={jacobian}")
    integral = 0
    fn = function(a=a_in, b=b_in)
    print(f"giving a_in, b_in to fnn yields: {fn}")
    print(f"the resulting lambda should yield 0.5 for any xi")
    print(f"testing: fn(xi=0) = {fn(xi=0)}")
    for p in range(0, len(gauss_points)): 
        # x = basis.map_xi_to_x(X0=x_lower, X1=x_upper, xi=gauss_points[p])
        pt = gauss_points[p]
        print(f'at {p}th point = {pt} with wt={gauss_weights[p]}')
        function_val = fn(xi=pt)
        print(f"...fn val currently = {function_val}")
        incr = function_val * gauss_weights[p]
        # incr = function_val * gauss_weights[p] * jacobian
        integral += incr
        print(f"added increment={incr} for curr val of {integral}")
    return integral



In [ ]:
# HERE'S HOW TO WRITE THE FUNCTION_IN FOR QUADRATURE:
def fnn(xi=0,a=0,b=0):
    return lambda xi: prodfn(a,b)
# fnn(a=1)(0) # --> returns 0.5 for all xi, but pos/neg dep on whether a=b
# THIS WAY IT TAKES NO ARGS GOING IN, BUT YOU CAN USE THE A,B GOING INTO THE QUADRATURE FUNCTION AND THE XI GQ PTS

integrate_by_quadrature2(function=fnn, x_lower=0, x_upper=1, n_quad=1, a_in=0, b_in=1)

giving a_in, b_in to fnn yields: <function fnn.<locals>.<lambda> at 0x116886680>
the resulting lambda should yield 0.5 for any xi
testing: fn(xi=0) = -0.5
at 0th point = 0.0 with wt=2.0
...fn val currently = -0.5
added increment=-1.0 for curr val of -1.0


-1.0

In [ ]:
np.array(([1., -1.], [-1., 1.]))

NameError: name 'np' is not defined